In [1]:
import os, sys, glob, argparse
import pandas as pd
import numpy as np
from tqdm import tqdm

import cv2
from PIL import Image
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")



In [2]:
train_path = glob.glob('./train_data/training_set/*/*')
np.random.shuffle(train_path)

In [3]:
n = len(train_path)
print(n)

20368


In [4]:
n_test = int(0.2 * n)
print(n_test)

4073


In [5]:
class_names = ['Asian Green Bee-Eater', 'Brown-Headed Barbet', 'Cattle Egret',
       'Common Kingfisher', 'Common Myna', 'Common Rosefinch',
       'Common Tailorbird', 'Coppersmith Barbet', 'Forest Wagtail',
       'Gray Wagtail', 'Hoopoe', 'House Crow', 'Indian Grey Hornbill',
       'Indian Peacock', 'Indian Pitta', 'Indian Roller',
       'Jungle Babbler', 'Northern Lapwing', 'Red-Wattled Lapwing',
       'Ruddy Shelduck', 'Rufous Treepie', 'Sarus Crane', 'White Wagtail',
       'White-Breasted Kingfisher', 'White-Breasted Waterhen']
len(class_names)

25

In [6]:
class XunFeiDataset(Dataset):
    def __init__(self, img_path, transform=None):
        self.img_path = img_path
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    
    def __getitem__(self, index):
        img = Image.open(self.img_path[index]).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        
        for idx, label in enumerate(class_names):
            if label in self.img_path[index]:
                label = idx
                break
        return img, torch.from_numpy(np.array(label).astype(int))
    
    def __len__(self):
        return len(self.img_path)

In [8]:
class XunFeiNet(nn.Module):
    def __init__(self):
        super(XunFeiNet, self).__init__()
        model = models.resnet18(True)
        num_features=model.fc.in_features
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(num_features, len(class_names))
        self.resnet = model
    
    def forward(self, img):
        out = self.resnet(img)
        return out

In [9]:
def train(train_loader, model, criterion, optimizer):
    model.train()
    train_loss = 0.0
    for i, (input, target) in enumerate(train_loader):
        input = input.to(device)
        target = target.to(device)

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print('Train loss', loss.item())
            
        train_loss += loss.item()
    
    return train_loss/len(train_loader)
            
def validate(val_loader, model, criterion):
    model.eval()
    
    val_acc = 0.0
    
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            input = input.to(device)
            target = target.to(device)

            # compute output
            output = model(input)
            loss = criterion(output, target)
            
            val_acc += (output.argmax(1) == target).sum().item()
            
    return val_acc / len(val_loader.dataset)

def predict(test_loader, model, criterion):
    model.eval()
    val_acc = 0.0
    
    test_pred = []
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            input = input.to(device)
            target = target.to(device)

            # compute output
            output = model(input)
            test_pred.append(output.data.cpu().numpy())
            
    return np.vstack(test_pred)

In [10]:
train_loader = torch.utils.data.DataLoader(
    XunFeiDataset(train_path[:-n_test],
    transforms.Compose([
                transforms.Resize(256),
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ])), batch_size=30, shuffle=True, num_workers=4, pin_memory=False
)

val_loader = torch.utils.data.DataLoader(
    XunFeiDataset(train_path[-n_test:],
    transforms.Compose([
                transforms.Resize(256),
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ])), batch_size=15, shuffle=False, num_workers=1, pin_memory=False
)

In [11]:
model = XunFeiNet()
model = model.to(device)
lr = 0.009
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr)

/opt/conda/envs/py_3.8/lib/python3.8/site-packages/torchvision-0.15.0a0+c206a47-py3.8-linux-x86_64.egg/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/opt/conda/envs/py_3.8/lib/python3.8/site-packages/torchvision-0.15.0a0+c206a47-py3.8-linux-x86_64.egg/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
for _  in range(40):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc  = validate(val_loader, model, criterion)
    print(train_loss, val_acc)
print("train done")

Train loss 3.388678789138794
Train loss 1.9799878597259521
Train loss 1.452166199684143
Train loss 0.9562422633171082
Train loss 0.9234104156494141
Train loss 0.8577191829681396
1.4012843057622804 0.7952369260986988
Train loss 0.7218042016029358
Train loss 0.5972363352775574
Train loss 0.954531729221344
Train loss 0.9149355292320251
Train loss 0.7557108402252197
Train loss 0.8184840679168701
0.6931042657507693 0.8347655290940339
Train loss 0.6208568215370178
Train loss 0.7852361798286438
Train loss 0.7617814540863037
Train loss 0.8572635054588318
Train loss 0.7386090755462646
Train loss 0.5402775406837463
0.5930682874339468 0.8502332433095998
Train loss 0.546901524066925
Train loss 0.3939149081707001
Train loss 0.26353999972343445
Train loss 0.4136384427547455
Train loss 0.7504159808158875
Train loss 0.8162568807601929
0.5448748861740836 0.8634912840657991
Train loss 0.4091905951499939
Train loss 0.26601487398147583
Train loss 0.4526286721229553
Train loss 0.53131103515625
Train loss 0

In [14]:
model = model.to('cpu')
torch.save(model.state_dict(), 'model_820.pt')

run.py 内容如下：

```python
import os, sys, glob, argparse
import pandas as pd
import numpy as np
from tqdm import tqdm

import cv2
from PIL import Image
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

# Check if GPU is available
device = torch.device("cpu")

class XunFeiDataset(Dataset):
    def __init__(self, img_path, transform=None):
        self.img_path = img_path
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    
    def __getitem__(self, index):
        img = Image.open(self.img_path[index]).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        
        label = 0
        
        return img, torch.from_numpy(np.array(label).astype(int))
    
    def __len__(self):
        return len(self.img_path)
    
class XunFeiNet(nn.Module):
    def __init__(self):
        super(XunFeiNet, self).__init__()
        model = models.resnet50(False)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(2048, 25)
        self.resnet = model
    
    def forward(self, img):
        out = self.resnet(img)
        return out
    
def predict(test_loader, model):
    model.eval()    
    test_pred = []
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            input = input.to(device)
            target = target.to(device)
            output = model(input)
            test_pred.append(output.data.cpu().numpy())
            
    return np.vstack(test_pred)


test_path = glob.glob('/work/data/birds-test-dataset/*')
test_path.sort()

test_loader = torch.utils.data.DataLoader(
    XunFeiDataset(test_path[:],
    transforms.Compose([
                transforms.Resize((224, 224)),
                # transforms.RandomResizedCrop(224),
                # transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ])), batch_size=30, shuffle=False, num_workers=1, pin_memory=False
)
model = XunFeiNet()
model.load_state_dict(torch.load('./model/model.pt'))

test_pred = predict(test_loader, model)
test_pred = test_pred.argmax(1)
# class_names = np.array(['Asian Green Bee-Eater', 'Brown-Headed Barbet', 'Cattle Egret',
#        'Common Kingfisher', 'Common Myna', 'Common Rosefinch',
#        'Common Tailorbird', 'Coppersmith Barbet', 'Forest Wagtail',
#        'Gray Wagtail', 'Hoopoe', 'House Crow', 'Indian Grey Hornbill',
#        'Indian Peacock', 'Indian Pitta', 'Indian Roller',
#        'Jungle Babbler', 'Northern Lapwing', 'Red-Wattled Lapwing',
#        'Ruddy Shelduck', 'Rufous Treepie', 'Sarus Crane', 'White Wagtail',
#        'White-Breasted Kingfisher', 'White-Breasted Waterhen'])
# test_pred = class_names[test_pred]

pd.DataFrame({
    'imageID': [x.split('/')[-1] for x in test_path],
    'label': test_pred
}).to_csv('/work/output/result.csv', index=None)
```